## Домашняя работа

Оценить расходы в цилиндр высокого давления и в конденсатор турбины без промежуточного переграва пара.

Заданные параметры:
* Номинальная электрическая мощность: 250 МВт
* Начальное давление свежего пара: 12.8 МПа
* Начальная температура свежено пара: 540 C
* Конечное давление пара: 6.9 кПа
* Температура питательной воды:  263 C
* Число отборов: 8
* Механическое КПД: 99.2%
* КПД Электрогенератора: 99%

In [1]:
import iapws
from iapws import IAPWS97 as gas

In [2]:
MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None

In [5]:
electrical_power = 250 * (10 ** 6)
p0 = 12.8 * MPa
t0 = 540
p_k = 6.9 * kPa
t_feed_water = 263
p_feed_water = 1.4 * p0
z = 8

mechanical_efficiency = 0.992
generator_efficiency = 0.99

In [6]:
point_0 = gas(P=p0*unit, T=to_kelvin(t0))
point_2 = gas(P=p_k * unit, s=point_0.s)

In [8]:
point_k_water = gas(P=pk * unit, x=0)
point_feed_water = gas(P=p_feed_water * unit, T=to_kelvin(t_feed_water))

In [9]:
numenator_without = point_2.T * (point_0.s - point_k_water.s)
denumenator_without = (point_0.h - point_k_water.h)
without_part = 1 - (numenator_without / denumenator_without)


numenator_infinity = point_2.T * (point_0.s - point_feed_water.s)
denumenator_infinity = (point_0.h - point_feed_water.h)
infinity_part = 1 - (numenator_infinity / denumenator_infinity)

ksi_infinity = 1 - (without_part / infinity_part)
ksi_infinity

0.14000220029703958

In [10]:
coeff = (point_feed_water.T - point_2.T) / (to_kelvin(374.2) - point_2.T)
coeff

0.668521593152601

In [32]:
ksi = 0.9 * ksi_infinity
ksi

0.12600198026733564

In [33]:
eff_num = (point_0.h - point_2.h) 
eff_denum = (point_0.h - point_2.h) + (point_0.h - point_k_water.h)

efficiency = (eff_num / eff_denum) * (1 / (1 - ksi))

In [34]:
estimated_heat_drop = efficiency * ((point_0.h - point_feed_water.h))
estimated_heat_drop

787.7118759864186

In [35]:
inlet_mass_flow = electrical_power / (estimated_heat_drop * 1000 * mechanical_efficiency * generator_efficiency)


In [36]:
condenser_mass_flow = (
    electrical_power /
    ((point_2.h - point_k_water.h) * 1000 * mechanical_efficiency * generator_efficiency) * ((1 / efficiency) - 1)
)

In [37]:
print("Массовый расход в турбину на входе", inlet_mass_flow)

Массовый расход в турбину на входе 323.1660639601928


In [38]:
print("Массовый расход в конденсатор:", condenser_mass_flow)

Массовый расход в конденсатор: 259.6832094020725
